In [ ]:
#########################################################
# hackathon Boavizta at Orange Gardens - May 24th, 2024 #
#########################################################

"""
______   _______  _______          _________ _______ _________ _______               _______  _______  _        _______ _________          _______  _       
(  ___ \ (  ___  )(  ___  )|\     /|\__   __// ___   )\__   __/(  ___  )    |\     /|(  ___  )(  ____ \| \    /\(  ___  )\__   __/|\     /|(  ___  )( (    /|
| (   ) )| (   ) || (   ) || )   ( |   ) (   \/   )  |   ) (   | (   ) |    | )   ( || (   ) || (    \/|  \  / /| (   ) |   ) (   | )   ( || (   ) ||  \  ( |
| (__/ / | |   | || (___) || |   | |   | |       /   )   | |   | (___) |    | (___) || (___) || |      |  (_/ / | (___) |   | |   | (___) || |   | ||   \ | |
|  __ (  | |   | ||  ___  |( (   ) )   | |      /   /    | |   |  ___  |    |  ___  ||  ___  || |      |   _ (  |  ___  |   | |   |  ___  || |   | || (\ \) |
| (  \ \ | |   | || (   ) | \ \_/ /    | |     /   /     | |   | (   ) |    | (   ) || (   ) || |      |  ( \ \ | (   ) |   | |   | (   ) || |   | || | \   |
| )___) )| (___) || )   ( |  \   /  ___) (___ /   (_/\   | |   | )   ( |    | )   ( || )   ( || (____/\|  /  \ \| )   ( |   | |   | )   ( || (___) || )  \  |
|/ \___/ (_______)|/     \|   \_/   \_______/(_______/   )_(   |/     \|    |/     \||/     \|(_______/|_/    \/|/     \|   )_(   |/     \|(_______)|/    )_)
                                                                                                                                                             
 _______ _________     _______  _______  _______  _        _______  _______      _______  _______  _______  ______   _______  _        _______               
(  ___  )\__   __/    (  ___  )(  ____ )(  ___  )( (    /|(  ____ \(  ____ \    (  ____ \(  ___  )(  ____ )(  __  \ (  ____ \( (    /|(  ____ \              
| (   ) |   ) (       | (   ) || (    )|| (   ) ||  \  ( || (    \/| (    \/    | (    \/| (   ) || (    )|| (  \  )| (    \/|  \  ( || (    \/              
| (___) |   | |       | |   | || (____)|| (___) ||   \ | || |      | (__        | |      | (___) || (____)|| |   ) || (__    |   \ | || (_____               
|  ___  |   | |       | |   | ||     __)|  ___  || (\ \) || | ____ |  __)       | | ____ |  ___  ||     __)| |   | ||  __)   | (\ \) |(_____  )              
| (   ) |   | |       | |   | || (\ (   | (   ) || | \   || | \_  )| (          | | \_  )| (   ) || (\ (   | |   ) || (      | | \   |      ) |              
| )   ( |   | |       | (___) || ) \ \__| )   ( || )  \  || (___) || (____/\    | (___) || )   ( || ) \ \__| (__/  )| (____/\| )  \  |/\____) |              
|/     \|   )_(       (_______)|/   \__/|/     \||/    )_)(_______)(_______/    (_______)|/     \||/   \__/(______/ (_______/|/    )_)\_______)              
                                                                                                                                                             
"""

In [ ]:
# dependencies
from PIL import Image
import os
import uuid
import json
from joblib import load
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.callbacks import EarlyStopping
import pathlib
import shutil
import math
import datetime
from typing import List, Tuple
import sys
from codecarbon import EmissionsTracker
from api.api import Algorithm, Dataset, Hardware, Measure, Report_Header, Report_Task, Report_System, Report_Software, Report_Region, Report_Hash, Report

In [ ]:
# image settings
import tensorflow.keras.backend as K
K.set_image_data_format('channels_last') # can be channels_first or channels_last. 

In [ ]:
# constants
TARGET_POPULATION_PER_CLASS = 5000
BATCH_SIZES = [ 16, 32, 64 ]
EPOCHS = 32
ACTIVATION = 'softmax'
CLASSIFIERS = [ 'baseline', 'resnet_50', 'inception_v3' ]
OPTIMIZERS = [ 'adam', 'adamw', 'nadam' ]
JOBLIB_FILE_EXTENSION = 'joblib'


In [ ]:
# params

"""ATTENTION PLEASE! Feel free to modify these values in order to avoid too long tasks. 
Please respect the values' ranges or enums.
"""
image_size = 100            # default - possible values from 96 to 192
batch_size = 16             # default - possible values in BATCH_SIZES
percent = 0.2               # default - possible values from 0.0 to 1.0
algorithm = 'baseline'      # default - possible values in CLASSIFIERS
early_stopping = True       # default - possible values True / False
optimizer = 'adam'          # default - possible values in OPTIMIZERS
learning_rate = 0.001       # default
report_folder = os.sep.join(['.', 'out'])


In [ ]:
# paths
report_filepath = os.sep.join([ report_folder, 'report-img_{}-batch_{}-pc_{}-alg_{}-es_{}-opt_{}-lr_{}.json'.format(str(image_size), str(batch_size), str(percent), str(algorithm), str(early_stopping), optimizer, str(learning_rate))])
cc_filename = 'emissions_codecarbon-img_{}-batch_{}-pc_{}-alg_{}-es_{}-opt_{}-lr_{}.csv'.format(str(image_size), str(batch_size), str(percent), str(algorithm), str(early_stopping), optimizer, str(learning_rate))
os.makedirs(report_folder, exist_ok=True)


In [ ]:
# report instantiation

"""ATTENTION! The main purpose of the Hackathon's discussions and achievements is here: fill automatically as many fields as possible 
in the report object below, either at runtime and/or thanks to code introspection techniques.
"""

"""The main object does not instantiate automatically the sub-object, because the JSON schema parser is currently sketchy. 
Not a big issue right now.
"""
report = Report()

"""The header part has a set of semi-static parameters to identify the publisher. 
Could be done quite trivially from a config file or environment variables.
"""
report.header = Report_Header()

"""Here the challenges begin! First, not all ML tasks manipulate datasets (think about an inference situation).
We use the datasets property typically if the tasks consists in a fit, a fine-tuning, a re-train, and the like.
Please notice there can be more than 1 dataset involved in a single task.
That said, it seems natural to retrieve the 'physionomy' of the dataset(s) at runtime.
...Which would mean a wrapper of the ML library (say, Keras, for example), capable to manage a kind of 'binding' 
to the energy monitoring object (say, CodeCarbon, for example). Or reciprocally.
...Alternatively, we could think about a system of decorators on the corresponding variables (to mean explicitly: 
'this variable contains the dataset from which we aim at retrieving properties').
"""
report.datasets = [ Dataset() ]

"""The measures property is where measures are stored. Once again, it is considered as a collection, 
in case the developer put concurrent energy monitoring solutions in the code (say, CodeCarbon vs Carbon AI).
No technical challenge since it is basically a copy-paste of the measures performed by the monitoring solution(s) in a dedicated object.
"""
report.measures = [ Measure() ]

"""Defining automatically the ML task (or even general IT task) being performed is a tricky part.
For most common ML libraries, some typical tasks may be identified.
...In particular, if the task, per se, is atomic. A training would be recognized easily by a method named 'fit(...)'.
In our case below, the ML task is the training of an image classifier. Line 'o = model.fit(...)' with function play(...) below.
How to handle properly this simple case? At runtime? Through offline introspection?
What about non-atomic tasks? Can some heuristic search help to make a smart guess to determine what the code is all about?
"""
report.task = Report_Task()

"""System information is clearly an easy data retrieval (platform.system() and that kind of things).
"""
report.system = Report_System()

"""Software environment, likewise, is an easy data retrival (sys.version_info and that kind of things).
"""
report.software = Report_Software()

"""Access to regional information is a native functionality of most energy monitoring libraries. 
There is accordingly no big challenge other than making a copy-paste from the library (say, CodeCarbon) to this property.
"""
report.region = Report_Region()

"""The hash is computed at the end of the process, when the report is completed. 
It embarks different purposes such as integrity and authentication of the publisher (if desired).
The hash may be computed on-the-fly by the process which sends the report to the open data repository.
"""
report.hash = Report_Hash()


In [ ]:
# setup energy monitor instance
print('starting energy tracker')
cc_tracker = EmissionsTracker(project_name='Hackathon Boavizta', measure_power_secs=10, save_to_file=True, output_dir=report_folder, output_file=cc_filename, log_level='error')
cc_tracker.start()


In [ ]:
# download and explore dataset
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = pathlib.Path(tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)).with_suffix('')
data_dir_s = str(data_dir)
emissions_start = cc_tracker.flush()
print('download, emissions start '+str(emissions_start))
shutil.rmtree(data_dir_s, ignore_errors=True)
data_dir = pathlib.Path(tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)).with_suffix('')


In [ ]:
# create folder for unused data
print('directories setup')
path_elements = data_dir_s.split(os.sep)
path_elements[-1] = '~'+path_elements[-1]
unused_dir_s = os.sep.join(path_elements)
shutil.rmtree(unused_dir_s, ignore_errors=True)
os.makedirs(unused_dir_s, exist_ok=True)
subfolders = [ f.name for f in os.scandir(data_dir_s) if f.is_dir() ]
for subfolder in subfolders:
    subfolder_path_elements = path_elements + [ subfolder ]
    subfolder_path = os.sep.join(subfolder_path_elements)
    os.makedirs(subfolder_path, exist_ok=True)
emissions_stop = cc_tracker.flush()
print('download, emissions stop '+str(emissions_stop))


In [ ]:
# images centered-square cropping
print('crop dataset')
for subfolder in subfolders:
    subfolder_content = list(os.scandir(os.sep.join([ data_dir_s, subfolder ])))
    for f in subfolder_content:
        img = Image.open(f.path)
        # keep image vertical despite possible undue transformation at loading time
        if img.size[0] > img.size[1]:
            img = img.transpose(Image.Transpose.ROTATE_270)
        half_square_side = int(min(img.size[0], img.size[1])/2)-1
        img = img.crop((int(img.size[0]/2)-half_square_side, int(img.size[1]/2)-half_square_side, int(img.size[0]/2)+half_square_side, int(img.size[1]/2)+half_square_side))
        img.save(fp=f.path)


In [ ]:
# frozen random numbers
print('load permanent random filenames')
initial_renames = load(filename=os.sep.join([ '.', '.'.join([ 'initial_renames', JOBLIB_FILE_EXTENSION ])]))


In [ ]:
# rename original files
index = 0
for subfolder in subfolders:
    data_dir_subfolder = os.sep.join([ data_dir_s, subfolder ])
    for f in list(os.scandir(data_dir_subfolder)):
        shutil.move(f.path, os.sep.join([ data_dir_subfolder, '.'.join([ initial_renames[index], 'jpg' ]) ]))
        index += 1


In [ ]:
# frozen image transformations
print('load permanent random transformations')
transformations = load(filename=os.sep.join([ '.', '.'.join([ 'transformations', JOBLIB_FILE_EXTENSION ])]))


In [ ]:
# extra images generation to get balanced classes
print('balance classes with extra images')
index_bis = 0
for subfolder in subfolders:
    subfolder_index = 0
    subfolder_content = list(os.scandir(os.sep.join([ data_dir_s, subfolder ])))
    class_population = len(subfolder_content)
    if class_population >= TARGET_POPULATION_PER_CLASS:
        continue
    for i in range(class_population, TARGET_POPULATION_PER_CLASS):
        img = Image.open(subfolder_content[subfolder_index].path)    
        # flip horizontally and vertically on a random basis
        if transformations[index_bis]['flip_lr']:
            img = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)
        if transformations[index_bis]['flip_tb']:
            img = img.transpose(Image.Transpose.FLIP_TOP_BOTTOM)
        half_square_side = int(transformations[index_bis]['zoom_factor'] * min(img.size[0], img.size[1])/3)
        img = img.rotate(angle=transformations[index_bis]['angle']*transformations[index_bis]['angle_sign'])
        img = img.crop((int(img.size[0]/2)-half_square_side, int(img.size[1]/2)-half_square_side, int(img.size[0]/2)+half_square_side, int(img.size[1]/2)+half_square_side))
        img.save(fp=os.sep.join([ unused_dir_s, subfolder, '.'.join([ initial_renames[index], 'jpg' ]) ]))
        index += 1
        index_bis +=1
        subfolder_index += 1
        if subfolder_index == len(subfolder_content):
            subfolder_index = 0

In [ ]:
# move per class a ratio of random files to reduce the number of items processed
def split_files_per_class(percent_used:float=1.0):
    for subfolder in subfolders:
        # first, reset the directories
        data_dir_subfolder = os.sep.join([ data_dir_s, subfolder ])
        unused_dir_subfolder = os.sep.join([ unused_dir_s, subfolder ])
        for f in os.scandir(unused_dir_subfolder):
            shutil.move(f.path, os.sep.join([ data_dir_subfolder, f.name ]))
            
        # proceed with moving percentage
        if percent_used > 0.999:
            continue
        class_image_count = len(list(data_dir.glob('{}{}*.jpg'.format(subfolder, os.sep))))
        num_to_move = int(math.floor((1.0-percent_used)*class_image_count))
        for i in range(0, num_to_move):
            random_file=os.listdir(data_dir_subfolder)[0]
            shutil.move(os.sep.join([ data_dir_subfolder, random_file ]),
                        os.sep.join([ unused_dir_subfolder, random_file ]))


In [ ]:
# reset after initial images generation
print('move extra images to active space')
split_files_per_class()


In [ ]:
# images count and folder info
image_count = len(list(data_dir.glob('*/*.jpg')))
print(str(image_count))


In [ ]:
# naive baseline classifier
def baseline(img_size:int, classes:int, activation:str='softmax') -> Model:
  return Sequential([
      Input((img_size, img_size, 3)),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(0.2),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(classes, activation=activation, name='outputs')
  ])
    

In [ ]:
# inception v3 model
def inception_v3(img_size:int, classes:int, activation:str='softmax') -> Model:
    model = tf.keras.applications.InceptionV3(
        include_top=True,
        input_tensor=None,
        weights=None,
        input_shape=(img_size, img_size, 3),
        pooling=None,
        classes=classes,
        classifier_activation=activation,
    )
    return model


In [ ]:
# resnet50 identity block
def identity_block(X: tf.Tensor, level: int, block: int, filters: List[int]) -> tf.Tensor:
    # layers will be called conv{level}_iden{block}_{convlayer_number_within_block}'
    conv_name = f'conv{level}_{block}' + '_{layer}_{type}'
    # unpack number of filters to be used for each conv layer
    f1, f2, f3 = filters
    # the shortcut branch of the identity block
    # takes the value of the block input
    X_shortcut = X
    # first convolutional layer (plus batch norm & relu activation, of course)
    X = Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1),
               padding='valid', name=conv_name.format(layer=1, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=1, type='bn'))(X)
    X = Activation('relu', name=conv_name.format(layer=1, type='relu'))(X)
    # second convolutional layer
    X = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1),
               padding='same', name=conv_name.format(layer=2, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=2, type='bn'))(X)
    X = Activation('relu')(X)
    # third convolutional layer
    X = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1),
               padding='valid', name=conv_name.format(layer=3, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=3, type='bn'))(X)
    # add shortcut branch to main path
    X = Add()([X, X_shortcut])
    # relu activation at the end of the block
    X = Activation('relu', name=conv_name.format(layer=3, type='relu'))(X)
    return X


In [ ]:
# resnet50 convolutional block
def convolutional_block(X: tf.Tensor, level: int, block: int, filters: List[int], s: Tuple[int,int,int]=(2, 2)) -> tf.Tensor:
     # layers will be called conv{level}_{block}_{convlayer_number_within_block}'
    conv_name = f'conv{level}_{block}' + '_{layer}_{type}'
    # unpack number of filters to be used for each conv layer
    f1, f2, f3 = filters
    # the shortcut branch of the convolutional block
    X_shortcut = X
    # first convolutional layer
    X = Conv2D(filters=f1, kernel_size=(1, 1), strides=s, padding='valid',
               name=conv_name.format(layer=1, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=1, type='bn'))(X)
    X = Activation('relu', name=conv_name.format(layer=1, type='relu'))(X)
    # second convolutional layer
    X = Conv2D(filters=f2, kernel_size=(3, 3), strides=(1, 1), padding='same',
               name=conv_name.format(layer=2, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=2, type='bn'))(X)
    X = Activation('relu', name=conv_name.format(layer=2, type='relu'))(X)
    # third convolutional layer
    X = Conv2D(filters=f3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name=conv_name.format(layer=3, type='conv'),
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=conv_name.format(layer=3, type='bn'))(X)
    # shortcut path
    X_shortcut = Conv2D(filters=f3, kernel_size=(1, 1), strides=s, padding='valid',
                        name=conv_name.format(layer='short', type='conv'),
                        kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=conv_name.format(layer='short', type='bn'))(X_shortcut)
    # add shortcut branch to main path
    X = Add()([X, X_shortcut])
    # nonlinearity
    X = Activation('relu', name=conv_name.format(layer=3, type='relu'))(X)
    return X


In [ ]:
# resnet50 model
def resnet_50(img_size:int, classes:int, activation:str='softmax') -> Model:
    input_size = (img_size, img_size, 3)
    # tensor placeholder for the model's input
    X_input = Input(input_size)
    ### Level 1 ###
    # padding
    X = ZeroPadding2D((3, 3))(X_input)
    # convolutional layer, followed by batch normalization and relu activation
    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2),
               name='conv1_1_1_conv',
               kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='conv1_1_1_nb')(X)
    X = Activation('relu')(X)
    ### Level 2 ###
    # max pooling layer to halve the size coming from the previous layer
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)
    # 1x convolutional block
    X = convolutional_block(X, level=2, block=1, filters=[64, 64, 256], s=(1, 1))
    # 2x identity blocks
    X = identity_block(X, level=2, block=2, filters=[64, 64, 256])
    X = identity_block(X, level=2, block=3, filters=[64, 64, 256])
    ### Level 3 ###
    # 1x convolutional block
    X = convolutional_block(X, level=3, block=1, filters=[128, 128, 512], s=(2, 2))
    # 3x identity blocks
    X = identity_block(X, level=3, block=2, filters=[128, 128, 512])
    X = identity_block(X, level=3, block=3, filters=[128, 128, 512])
    X = identity_block(X, level=3, block=4, filters=[128, 128, 512])
    ### Level 4 ###
    # 1x convolutional block
    X = convolutional_block(X, level=4, block=1, filters=[256, 256, 1024], s=(2, 2))
    # 5x identity blocks
    X = identity_block(X, level=4, block=2, filters=[256, 256, 1024])
    X = identity_block(X, level=4, block=3, filters=[256, 256, 1024])
    X = identity_block(X, level=4, block=4, filters=[256, 256, 1024])
    X = identity_block(X, level=4, block=5, filters=[256, 256, 1024])
    X = identity_block(X, level=4, block=6, filters=[256, 256, 1024])
    ### Level 5 ###
    # 1x convolutional block
    X = convolutional_block(X, level=5, block=1, filters=[512, 512, 2048], s=(2, 2))
    # 2x identity blocks
    X = identity_block(X, level=5, block=2, filters=[512, 512, 2048])
    X = identity_block(X, level=5, block=3, filters=[512, 512, 2048])
    # Pooling layers
    X = AveragePooling2D(pool_size=(2, 2), name='avg_pool')(X)
    # Output layer
    X = Flatten()(X)
    X = Dense(classes, activation=activation, name='fc_' + str(classes),
              kernel_initializer=glorot_uniform(seed=0))(X)
    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    return model


In [ ]:
# unitary model training and validation
def play(cc_tracker:EmissionsTracker=None, model_name:str='baseline', batch_size:int=32, img_size:int=80, epochs:int=256, percent_used:float=1.0, activation:str='softmax', es:bool=True, optimizer:str='adam', learning_rate:float=0.001, report:dict=None):
    session_id = str(uuid.uuid4())

    try:
        print('flush CodeCarbon tracker')
        emissions_start = cc_tracker.flush() if cc_tracker is not None else 1.0

        # tailor dataset according to percent desired
        print('split files')
        split_files_per_class(percent_used)

        # train
        print('train dataset setup')
        train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset='training',
        seed=123,
        image_size=(img_size, img_size),
        batch_size=batch_size)

        # validation
        print('validation dataset setup')
        val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset='validation',
        seed=123,
        image_size=(img_size, img_size),
        batch_size=batch_size)
        
        # classes
        class_names = train_ds.class_names
        num_classes = len(class_names)
        print('number of classes: '+str(num_classes))

        # normalize
        print('normalization')
        normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
        norm_train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
        norm_val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
        image_batch, labels_batch = next(iter(norm_train_ds))

        # setup autotune
        print('autotune')
        AUTOTUNE = tf.data.AUTOTUNE
        norm_train_ds = norm_train_ds.cache().prefetch(buffer_size=AUTOTUNE)
        norm_val_ds = norm_val_ds.cache().prefetch(buffer_size=AUTOTUNE)

        # model
        print('model build')
        if model_name not in CLASSIFIERS:
            print('unknown model')
            return None
        model_func_call = '{}(img_size={}, classes={}, activation="{}")'.format(model_name, str(img_size), str(num_classes), activation)
        print('model function call is '+model_func_call)
        op = None
        if optimizer not in OPTIMIZERS:
            print('unknown optimzer')
            return None
        beta_1 = max(0.5, 1.0-100*learning_rate)
        beta_2 = max(0.95, 1.0-learning_rate)
        if optimizer == 'adam':
            op = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2)
        elif optimizer == 'adamw':
            op = tf.keras.optimizers.AdamW(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2)
        elif optimizer == 'nadam':
            op = tf.keras.optimizers.Nadam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2)
        model = eval(model_func_call)
        model.compile(optimizer=op,
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                        metrics=['accuracy'])
        early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=int(max(2, math.floor(epochs*0.1))), verbose=1, mode='auto')

        """The main task is HERE!
        """
        o = model.fit(norm_train_ds, validation_data=norm_val_ds, epochs=epochs, callbacks=[early]) if es else model.fit(norm_train_ds, validation_data=norm_val_ds, epochs=epochs)
        
        # visualize new training results
        acc = o.history['accuracy']
        val_acc = o.history['val_accuracy']
        loss = o.history['loss']
        val_loss = o.history['val_loss']
        stopped_epoch = epochs if early.stopped_epoch <= 0 else early.stopped_epoch

    except Exception as err:
        """TODO
        You probably want to do something with the report when things went wrong...
        """
        pass

    """TODO
    A certain number of properties of the report should be populated here.
    Pay attention to the fact that the code must stay generic. It is NOT aware of the roles played by each variable.
    """
    
    return model


In [ ]:
# main task
print('starting play model {} size {} epochs {} percent {} activation {} early stopping {}'.format(algorithm, str(image_size), str(EPOCHS), str(percent), ACTIVATION, str(early_stopping)))
try:

    play(
        cc_tracker=cc_tracker, 
        model_name=algorithm, 
        batch_size=batch_size, 
        img_size=image_size, 
        epochs=EPOCHS, 
        percent_used=percent, 
        activation=ACTIVATION, 
        es=early_stopping,
        optimizer=optimizer,
        learning_rate=learning_rate,
        report=report)

except Exception as err:
    print('exception raised: '+str(err))

print('ending play model')
split_files_per_class() # reset directories' content
cc_tracker.stop() 

"""TODO
Persist the report somewhere, for further propagation to the open data repository.
ENJOY!
"""